<a href="https://colab.research.google.com/github/kmAyush/Video-Dubbing-with-Lip-Synchronization/blob/main/Text%20Translation/Get_trans_text_BDGL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gdown

In [ ]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import gdown

In [ ]:
url = "https://drive.google.com/file/d/1-3yAkoJtKfNyr5bHh2qayuQVOeMYTPVO/view?usp=drive_link"
!mkdir -p tokenizer
id = url.split('/')[-2]
file_url = f'https://drive.google.com/uc?id={id}'
destination_location = "/content/tokenizer/"
gdown.download(file_url , destination_location, quiet=False )

Downloading...
From: https://drive.google.com/uc?id=1-3yAkoJtKfNyr5bHh2qayuQVOeMYTPVO
To: /content/tokenizer/english.pkl
100%|██████████| 741k/741k [00:00<00:00, 98.0MB/s]


'/content/tokenizer/english.pkl'

In [ ]:
url = "https://drive.google.com/file/d/1-ACb3okGOeYACwCofyOIVla6udM5gVnl/view?usp=drive_link"
!mkdir -p tokenizer
id = url.split('/')[-2]
file_url = f'https://drive.google.com/uc?id={id}'
destination_location = "/content/tokenizer/"
gdown.download(file_url , destination_location, quiet=False )

Downloading...
From: https://drive.google.com/uc?id=1-ACb3okGOeYACwCofyOIVla6udM5gVnl
To: /content/tokenizer/hindi.pkl
100%|██████████| 1.08M/1.08M [00:00<00:00, 122MB/s]


'/content/tokenizer/hindi.pkl'

In [ ]:
url = "https://drive.google.com/file/d/1-Dd_CTPUDG40FmGXUJQ3G7SfMFhTPgJL/view?usp=sharing"
!mkdir -p model
id = url.split('/')[-2]
file_url = f'https://drive.google.com/uc?id={id}'
destination_location = "/content/model/"
gdown.download(file_url , destination_location, quiet=False )

Downloading...
From (original): https://drive.google.com/uc?id=1-Dd_CTPUDG40FmGXUJQ3G7SfMFhTPgJL
From (redirected): https://drive.google.com/uc?id=1-Dd_CTPUDG40FmGXUJQ3G7SfMFhTPgJL&confirm=t&uuid=fd55db81-97e2-4025-ba12-9d7b8e10f01b
To: /content/model/bidir_word_clip_model.pt
100%|██████████| 404M/404M [00:02<00:00, 197MB/s]


'/content/model/bidir_word_clip_model.pt'

In [ ]:
file_path = '/content/tokenizer/english.pkl'

with open(file_path, 'rb') as file:
   tokenizer_eng = pickle.load(file)


file_path = '/content/tokenizer/hindi.pkl'

with open(file_path, 'rb') as file:
   tokenizer_hin = pickle.load(file)

eng_vocab = tokenizer_eng.get_vocab()
hin_vocab = tokenizer_hin.get_vocab()


In [ ]:
class AttentionModule(torch.nn.Module):
    """ Implements an attention module """
    def __init__(self, input_size):
        """ Initializes the attention module.
            Feel free to declare any parameters as required. """

        super(AttentionModule, self).__init__()

        self.W_key = nn.Linear((2*input_size), input_size)
        self.W_query = nn.Linear(input_size, input_size)
        self.W_value =nn.Linear((2*input_size), input_size)


    def forward(self, encoder_outputs, decoder_hidden_state):
        """ Performs a forward pass over the module, computing attention scores for inputs.

        Args:
            encoder_outputs (torch.Tensor): Output representations from the encoder, of shape [batch_size?, src_seq_len, output_dim].
            decoder_hidden_state (torch.Tensor): Hidden state from the decoder at current time step, of appropriate shape as per RNN unit (with optional batch dim).

        Returns:
            torch.Tensor: Attentions scores for given inputs, of shape [batch_size?, 1, src_seq_len]
        """
        keys = self.W_key(encoder_outputs)  # batch,seq_length, hidden_size
        values = self.W_value(encoder_outputs)   # batch,seq_length, hidden_size

        # decoder_hidden_state ==  num_layer,N, hid_size

        num_layers = decoder_hidden_state.size(0)
        decoder_hidden_state = decoder_hidden_state[num_layers-1].unsqueeze(1)  # N , 1, hidden
        query = self.W_query(decoder_hidden_state)  #  batch,1 ,hidden_size
        query = query.transpose(1, 2)  # (batch size, hidden size, 1)

        weights = torch.matmul(keys, query)  # batch , sequence, 1
        weights = F.softmax(weights, dim = 1)   # (batch, sequence,  1)

        context_vector = weights * values  # N, seq_length , hidden

        context_vector = torch.sum(context_vector, dim = 1, keepdim = True) # N, 1, hidden

        return context_vector, weights

In [ ]:
## ==== BEGIN EVALUATION PORTION

class RNNEncoderDecoderLMWithAttention(torch.nn.Module):
    """ Implements an Encoder-Decoder network, using RNN units, augmented with attention. """

    # Feel free to add additional parameters to __init__
    def __init__(self,src_vocab_size, tgt_vocab_size, embd_dims, hidden_size, num_layers=1, dropout=0.1, teacher_force_ratio = 1 ):
        """ Initializes the encoder-decoder network, implemented via RNNs.

        Args:
            src_vocab_size (int): Source vocabulary size.
            tgt_vocab_size (int): Target vocabulary size.
            embd_dims (int): Embedding dimensions.
            hidden_size (int): Size/Dimensions for the hidden states.
        """

        super(RNNEncoderDecoderLMWithAttention, self).__init__()

        self._dummy_param = torch.nn.Parameter(torch.Tensor(0), requires_grad=False)

        self.src_vocab_size = src_vocab_size
        self.tgt_vocab_size = tgt_vocab_size

        self.num_layers = num_layers
        self.hidden_size = hidden_size

        self.dropout_enc = nn.Dropout(dropout)
        self.dropout_dec = nn.Dropout(dropout)
        self.teacher_force_ratio = teacher_force_ratio
        self.enc_pad_index = 0

        self.tgt_pad_index = 0
        self.tgt_start_index = 1
        self.tgt_end_index = 2

        self.attension_module = AttentionModule(hidden_size)

        self.encoder_embedding = torch.nn.Embedding(src_vocab_size, embd_dims)
        self.encoder_lstm = nn.LSTM(embd_dims, hidden_size,num_layers = num_layers , batch_first = True, dropout = dropout, bidirectional=True)

        self.decoder_embedding = torch.nn.Embedding(tgt_vocab_size, embd_dims)
        self.decoder_lstm = nn.LSTM(embd_dims+hidden_size, hidden_size,num_layers = num_layers, batch_first = True, dropout = dropout)
        self.decoder_output_fc = nn.Linear(hidden_size, embd_dims)
        self.dec_out_embed = nn.Linear(embd_dims,tgt_vocab_size, bias = False)

        self.dec_out_embed.weight = self.decoder_embedding.weight

    @property
    def device(self):
        return self._dummy_param.device

    def forward(self, inputs, decoder_inputs=None, decoder_hidden_state=None, output_attention=False ):
        """ Performs a forward pass over the encoder-decoder network.

            Accepts inputs for the encoder, inputs for the decoder, and hidden state for
                the decoder to continue generation after the given input.

        Args:
            inputs (torch.Tensor): tensor of shape [batch_size?, src_seq_len]
            decoder_inputs (torch.Tensor): Decoder inputs, as tensor of shape [batch_size?, 1]
            decoder_hidden_state (any): tensor to represent decoder hidden state from time step T-1.
            output_attention (bool): If true, this function should also return the
                associated attention weights for the time step, of shape [batch_size?, 1, src_seq_len].

        Returns:
            tuple[torch.Tensor, any]: output from the decoder, and associated hidden state for the next step.

            Decoder outputs should be log probabilities over the target vocabulary.

        Example:
        >>> model = RNNEncoderDecoderWithAttention(*args, **kwargs)
        >>> output, hidden = model(..., output_attention=False)
        >>> output, hidden, attn_weights = model(..., output_attention=True)
        """

        batch_size = inputs.size(0)
        seq_length = decoder_inputs.size(1) if decoder_inputs != None else inputs.size(1)
        encoder_input_embed = self.encoder_embedding(inputs)
        encoder_output, (encoder_hidden ,cell_encoder )= self.encoder_lstm(encoder_input_embed)

        decoder_hidden = encoder_hidden[:self.num_layers,:,:]
        decoder_cell = cell_encoder[:self.num_layers,:,:]
        decoder_input_test = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(self.tgt_start_index)


        pred_output_prob = []
        weights  = []
        for i in range(1,seq_length):

            decoder_input_embed = self.decoder_embedding(decoder_input_test)
            context_vector, wts = self.attension_module(encoder_output, decoder_hidden)
            decoder_input_lstm = torch.cat((decoder_input_embed, context_vector), dim = 2)

            dec_output, (decoder_hidden,decoder_cell) = self.decoder_lstm(decoder_input_lstm, (decoder_hidden,decoder_cell))
            out_tgt = self.decoder_output_fc(dec_output)
            out_tgt_embd = self.dec_out_embed(out_tgt)
            pred_output = F.log_softmax(out_tgt_embd, dim = -1)

            if decoder_inputs != None:
              teacher_frc = torch.rand(1, 1)
              if teacher_frc < self.teacher_force_ratio:
                 decoder_input_test = decoder_inputs[:, i].unsqueeze(1)

              else:
                 _, topi = pred_output.topk(1)
                 decoder_input_test = topi.squeeze(-1).detach()
            else:
               _, topi = pred_output.topk(1)
               decoder_input_test = topi.squeeze(-1).detach()

            pred_output_prob.append(pred_output)
            wts = wts.squeeze(0).squeeze(1)
            weights.append(wts)


        pred_output_prob = torch.cat(pred_output_prob, dim = 1)
        weights = torch.stack(weights)


        return pred_output_prob,weights

In [ ]:
def remove_paddings(sequence,pad_token):
  while(sequence[-1] == pad_token):
    sequence = sequence[:-1]

  return sequence

def rnn_greedy_generate(model, seq_x, max_length=5):
    """ Given a source string, translate it to the target language using the trained model.
        This function should perform greedy sampling to generate the results.

    Args:
        model (nn.Module): RNN Type Encoder-Decoder Model
        seq_x (str): Input string to translate.
        src_tokenizer (Tokenizer): Source language tokenizer.
        tgt_tokenizer (Tokenizer): Target language tokenizer.
        max_length (int): Maximum length of the target sequence to decode.

    Returns:
        str: Generated string for the given input in the target language.
    """

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    input_length = len(seq_x)

    model.eval()

    input_tokens = torch.tensor(seq_x).unsqueeze(0)
    input_tokens = input_tokens.to(device)

    output_probs,_ = model(input_tokens)
    tgt_tokens = torch.argmax(output_probs, dim = 2).squeeze(0).tolist()

    tgt_tokens = remove_paddings(tgt_tokens,0)
    trans_seq = tokenizer_hin.decode(tgt_tokens[:-1])

    return trans_seq

In [ ]:
rnn_enc_dec_attn_params = {
    'src_vocab_size': len(eng_vocab),
    'tgt_vocab_size': len(hin_vocab),
    'embd_dims'     : 400,
    'hidden_size'   : 800,
    'dropout'       : 0.3,
    'num_layers'    : 4,
    'teacher_force_ratio' : 0.6
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_path = '/content/model/bidir_word_clip_model.pt'
model_state_dict = torch.load(model_path,map_location = device)

model3 = RNNEncoderDecoderLMWithAttention(**rnn_enc_dec_attn_params)
model3.load_state_dict(model_state_dict)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model3 = model3.to(device)


In [ ]:
def generate_trans_text(model, eng_text):
  seq_x = tokenizer_eng.encode("<SOS>").ids+ tokenizer_eng.encode(eng_text).ids + tokenizer_eng.encode("<EOS>").ids
  trans_text = rnn_greedy_generate(model,seq_x)
  return trans_text
